# Prepare Exercice 1

In [1]:
import pandas as pd
from Fast_connectCloud import connector
import plotly.express as px
import numpy as np
import plotly.express as px
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

In [2]:
gs = connector.open_connection(online_connection = False,
                              path_credential = '/Users/thomas/Google Drive/Projects/Data_science/Google_code_n_Oauth/Client_Oauth/Google_auth/')

service = gs.connect_remote('GCP')

Service account storage and Bigquery are now connected. 
Service account storage is stored as <google.cloud.storage.client.Client object at 0x1a237ea4a8> and accessible with "Storage_account" 
Service account Bigquery is stored as <google.cloud.bigquery.client.Client object at 0x1a237ea668> and accessible with "bigquery_account"


In [3]:
from GoogleDrivePy.google_platform import connect_cloud_platform
project = 'valid-pagoda-132423'
gcp = connect_cloud_platform.connect_console(project = project, 
                                             service_account = service['GoogleCloudP'])    
query = (
          "SELECT * "
            "FROM Business.Vodafone "

        )

df_final = gcp.upload_data_from_bigquery(query = query, location = 'US')
df_final.sort_values(by = ['Customer_Name']).head()

,Customer_Number,Customer_Name,Country_Scrub,Country_name,WB,Inbound_Region,str,Essbase_Affiliation,VGE_Account,Source,...,Mobility_vodafone,Mobility_potential,Unified_Comms_vodafone,Unified_Comms_potential,Total_revenue_vodafone,Final_TAM,corrected_TAM,Total_market,Whitespace,UK
17333,1,Customer 1,Portugal,Portugal,PRT,Southern Europe,Professional Services,OpCo,True,ThirdParty,...,1656.769326,50156.075654,0.0,2285.310043,3834.291422,2.424632e+05,2.424632e+05,2.462975e+05,2.386289e+05,ROW
8513,1,Customer 1,Switzerland,Switzerland,CHE,Central Europe,Professional Services,None,True,ThirdParty,...,0.000000,0.000000,0.0,11013.613971,0.000000,8.928061e+05,8.928061e+05,8.928061e+05,8.928061e+05,ROW
11582,1,Customer 1,Brazil,Brazil,BRA,Americas,Professional Services,None,True,ThirdParty,...,0.000000,0.000000,0.0,0.000000,0.000000,8.239268e+05,8.239268e+05,8.239268e+05,8.239268e+05,ROW
17697,1,Customer 1,United States,United States,USA,Americas,Professional Services,MNS_Entity,True,ThirdParty,...,0.000000,0.000000,0.0,0.000000,31411.771429,1.740917e+07,1.740917e+07,1.744058e+07,1.737776e+07,ROW
14128,1,Customer 1,Finland,Finland,FIN,Northern Europe,Professional Services,None,True,ThirdParty,...,0.000000,0.000000,0.0,4872.040785,0.000000,4.257417e+05,4.257417e+05,4.257417e+05,4.257417e+05,ROW


In [4]:
df_final['str'].unique()

array(['Consumer Goods', 'Energy & Utilities', 'Financial Services',
       'Insurance', 'Manufacturing', 'Professional Services',
       'Technology', 'Transportation & Logistics', 'Travel & Tourism',
       'Automotive', 'Construction', 'Healthcare',
       'Media & Entertainment', 'Government & Public Sector',
       'Conglomerate', 'IPS', 'Agriculture', 'PRIOR-VGE'], dtype=object)

In [5]:
df_final = df_final.replace({'str': {"Consumer Goods": "A",
       'Energy & Utilities': "B",
       'Financial Services':"C",
'Insurance':"D",
       'Manufacturing':"E",
       'Professional Services':"F",
       'Technology':"G",
       'Transportation & Logistics':"H",
       'Travel & Tourism':"I",
       'Automotive':"J", 
       'Construction':"K",
       'Healthcare':"L",
       'Media & Entertainment':"M",
       'Government & Public Sector':"N",
       'Conglomerate':"O",
       'IPS':"P",
       'Agriculture':"Q",
       'PRIOR-VGE':"R"
}
})

In [6]:
list(df_final)

['Customer_Number',
 'Customer_Name',
 'Country_Scrub',
 'Country_name',
 'WB',
 'Inbound_Region',
 'str',
 'Essbase_Affiliation',
 'VGE_Account',
 'Source',
 'Fixed_vodafone',
 'Fixed_potential',
 'Cloud_Hosting_vodafone',
 'Cloud_Hosting_potential',
 'IoT_vodafone',
 'IoT_potential',
 'Mobility_vodafone',
 'Mobility_potential',
 'Unified_Comms_vodafone',
 'Unified_Comms_potential',
 'Total_revenue_vodafone',
 'Final_TAM',
 'corrected_TAM',
 'Total_market',
 'Whitespace',
 'UK']

In [7]:
df_final = df_final.drop(columns = ["Customer_Number", "Country_Scrub", "Essbase_Affiliation",
                        "VGE_Account","Source", "Final_TAM", "UK", 
                         "Total_market", "Whitespace"])

In [8]:
list(df_final)

['Customer_Name',
 'Country_name',
 'WB',
 'Inbound_Region',
 'str',
 'Fixed_vodafone',
 'Fixed_potential',
 'Cloud_Hosting_vodafone',
 'Cloud_Hosting_potential',
 'IoT_vodafone',
 'IoT_potential',
 'Mobility_vodafone',
 'Mobility_potential',
 'Unified_Comms_vodafone',
 'Unified_Comms_potential',
 'Total_revenue_vodafone',
 'corrected_TAM']

In [14]:
rename = ['Customer_Name',
 'Country_name',
 'WB',
 'Inbound_Region',
 'str',
 'A_reference',
 'A_market',
 'B_reference',
 'B_market',
 'C_reference',
 'C_market',
 'D_reference',
 'D_market',
 'E_reference',
 'E_market',
 'Total_reference',
 'Total_market']

df_final = df_final.rename(columns={"Customer_Name": "ID",
                         "Country_name": "Country_name",
                         "WB": "country_ref",
                          "Inbound_Region": "Region",
                          "str": "industry",
                          "Fixed_vodafone": "A_reference",
                          "Fixed_potential": "A_market",
                          "Cloud_Hosting_vodafone": "B_reference",
                          "Cloud_Hosting_potential": "B_market",
                          "IoT_vodafone": "C_reference",
                          "IoT_potential": "C_market",
                          "Mobility_vodafone": "D_reference",
                          "Mobility_potential": "D_market",
                          "Unified_Comms_vodafone": "E_reference",
                          "Unified_Comms_potential": "E_market",
                         "Total_revenue_vodafone": "Total_reference",
                         "corrected_TAM": "Total_market",
                        })
df_final.iloc[:,5:] = df_final.iloc[:,5:].transform([np.sqrt])/1000

In [15]:
df_final.head()

,ID,Country_name,country_ref,Region,industry,A_reference,A_market,B_reference,B_market,C_reference,C_market,D_reference,D_market,E_reference,E_market,Total_reference,Total_market
0,Customer 620,Australia,AUS,Asia Pacific,A,0.0,0.007258,0.0,0.010424,0.0,0.021051,0.0,0.001752,0.0,0.005433,0.0,0.021454
1,Customer 1070,Australia,AUS,Asia Pacific,A,0.0,0.009222,0.0,0.018054,0.0,0.021051,0.0,0.009690,0.0,0.008571,0.0,0.023859
2,Customer 25,Australia,AUS,Asia Pacific,A,0.0,0.012397,0.0,0.012396,0.0,0.002533,0.0,0.007179,0.0,0.005884,0.0,0.015037
3,Customer 370,Australia,AUS,Asia Pacific,B,0.0,0.012402,0.0,0.032244,0.0,0.009523,0.0,0.016637,0.0,0.016271,0.0,0.033501
4,Customer 597,Australia,AUS,Asia Pacific,C,0.0,0.014409,0.0,0.015047,0.0,0.010695,0.0,0.009601,0.0,0.010526,0.0,0.018923


In [16]:
df_final.to_csv(
	'dataPandasClass.gz',
	sep=',',
	header=True,
	index=False,
	chunksize=100000,
	compression='gzip',
	encoding='utf-8')

### Then upload to GCS
bucket_name = 'machine_learning_teaching'
destination_blob_name = 'Business_data'
source_file_name = 'dataPandasClass.gz'
gcp.upload_blob(bucket_name, destination_blob_name, source_file_name)

File dataPandasClass.gz uploaded to Business_data.


# Prepare Exercice 2